In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

In [49]:
# Data Array: (Distances, AWG Driving Voltage, Rx Voltage) Data Points
    # Array Position -> Data Point:
        # Distances              0 -> 5 mm,          1 -> 10 mm,         , ...
        # AWG Driving Voltages   0 -> 300 mV,        1 -> 400 mV,        , ...
        # Rx Voltage             0 -> V_Rx @ t=-100, 1 -> V_Rx @ t=-100+δ, ...

data = np.empty((3, 17, 5004)) 

distances = sorted(listdir("../measurements/csv"))
i = 0
for distance in distances:
    awg_voltages = sorted(listdir("../measurements/csv/" + distance))
    j = 0
    for awg_voltage in awg_voltages:
        measurements = sorted(listdir("../measurements/csv/" + distance + "/" + awg_voltage))
        k = 0
        for measurement in measurements:
            csv_data = pd.read_csv("../measurements/csv/" + distance + "/" + awg_voltage + "/" + measurement, names=["Time", "Rx Voltage (mV)"])[2:].drop(["Time"], axis=1).reset_index(drop=True).astype("float64").to_numpy().flatten()
            data[i][j][5004*k:5003*(k+1)-1]
            k += 1
        j += 1
    i += 1

52.17749